# Pré-Processamento: SparkSQL

### Passo 1: Iniciar Sessão Spark

In [ ]:
import os
import sys

# Limpar ambiente Spark completamente
def reset_spark():
    for var in ['spark', 'sc', 'sqlContext']:
        if var in globals():
            try:
                del globals()[var]
            except:
                pass
    
    # Limpar módulos
    modules = [k for k in sys.modules.keys() if 'pyspark' in k]
    for m in modules:
        if m in sys.modules:
            del sys.modules[m]

reset_spark()

from pyspark.sql import SparkSession

try:
    spark.stop()
except NameError:
    print("SparkContext not defined")

# local mode
# spark = SparkSession.builder \
#             .appName("Spark SQL") \
#             .master("local[*]") \
# 	    	.config("spark.jars.packages", "org.xerial.snappy:snappy-java:1.1.10.1") \
# 	    	.getOrCreate()

# cluster mode
spark = SparkSession.builder \
           .appName("Spark SQL basic example") \
           .master("spark://spark-master-g7:7077") \
	    	.config("spark.some.config.option", "some-value") \
	    	.getOrCreate()

### Passo 2: Escolha trabalhar com o dataset completo ou usando os dados amostrais

- Atenção: somente execute o conjunto de células desejado.

#### Dataset Completo

In [ ]:
from pathlib import Path
import gdown
import zipfile

ROOT_DIR = Path.cwd().parent
data_path = ROOT_DIR / "data"
data_raw_path = data_path / "data_raw"
zip_path = data_path / "data_raw.zip"
file_id = "1wLFeP8SPEuq_Ac6cPZTFE4PlR3bYdMRP"
download_url = f"https://drive.google.com/uc?export=download&id={file_id}"

if not data_path.exists() or not data_raw_path.exists():
    print(f"Pasta {data_raw_path} não encontrada. Baixando os dados...")
    
    data_path.mkdir(parents=True, exist_ok=True)

    gdown.download(download_url, str(zip_path), quiet=False)

    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(data_path)

    zip_path.unlink()

    print("Download e extração concluídos.")
else:
    print(f"Pasta {data_raw_path} encontrada.")

In [ ]:
import glob

arquivos_detalhados = glob.glob("../data/data_raw/**/*_DETAIL_*.csv", recursive=True)

arquivos_principais = [
    arq for arq in glob.glob("../data/data_raw/**/*.csv", recursive=True)
    if "_DETAIL_" not in arq
]

#### Dados amostrais

In [ ]:
arquivos_detalhados = "../datasample/RESTRICAO_COFF_EOLICA_DETAIL_2025_05_amostra.csv"

arquivos_principais = "../datasample/RESTRICAO_COFF_EOLICA_2025_05_amostra.csv"

### Passo 3: Pré-processamento

In [ ]:
df_detalhado = spark.read.option("delimiter", ";").option("header", True).option("inferSchema", True).csv(arquivos_detalhados)

df_principal = spark.read.option("delimiter", ";").option("header", True).option("inferSchema", True).csv(arquivos_principais)

In [ ]:
df_detalhado.show()
df_detalhado.count()

In [ ]:
df_principal.show()
df_principal.count()

In [ ]:
df_detalhado.write.option("compression", "uncompressed").mode("overwrite").parquet("../data/data_cleared/dados_detalhados.parquet")

In [ ]:
df_principal.write.option("compression", "uncompressed").mode("overwrite").parquet("../data/data_cleared/dados_principais.parquet")

In [ ]:
# Aplicar transformações aos dados principais antes de salvar
from pyspark.sql.functions import col, when, year, month, hour

print("Aplicando transformações aos dados...")

# Calcular constrained-off
df_principal = df_principal.withColumn(
    "constrained_off",
    when(col("val_disponibilidade") - col("val_geracao") > 0,
         col("val_disponibilidade") - col("val_geracao"))
    .otherwise(0)
)

# Calcular percentual de constrained-off
df_principal = df_principal.withColumn(
    "percentual_constrained",
    when(col("val_disponibilidade") > 0,
         (col("constrained_off") / col("val_disponibilidade")) * 100)
    .otherwise(0)
)

# Adicionar colunas temporais
df_principal = df_principal.withColumn("ano", year("din_instante"))
df_principal = df_principal.withColumn("mes", month("din_instante"))
df_principal = df_principal.withColumn("hora", hour("din_instante"))

print("✅ Transformações aplicadas")
print("Colunas adicionadas: constrained_off, percentual_constrained, ano, mes, hora")

In [ ]:
### 📊 Para Análises de Performance
**Nota**: Para análises comparativas de performance entre CSV e Parquet e benchmarking completo, utilize o notebook `Performance-Benchmark.ipynb`.